In [19]:
# Import
import os
import json
import shutil
import cv2
from sklearn.model_selection import train_test_split

### Installation von YOLOv5 mit Git und pip

In [ ]:
!git clone "https://github.com/ultralytics/yolov5.git"
%cd yolov5
%pip install -qr requirements.txt
%cd ..

### Kopieren von Bildern in den YOLOv5-Dataset-Ordner und Konvertieren von JSON-Daten in das YOLOv5-Format (txt)


In [20]:
from sklearn.model_selection import train_test_split
import shutil
import os

# Pfad zu den JSON-Daten und Bildern
json_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical mask/annotations" 
image_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images"  

# Pfad zu den YOLOv5 Dataset-Ordnern
labels_output_dir = "yolov5/dataset/labels"  
image_output_dir = "yolov5/dataset/images"  

# Bilder und Annotationen einlesen
images = [os.path.join(image_dir, x) for x in os.listdir(image_dir)]
annotations = [os.path.join(json_dir, x) for x in os.listdir(json_dir) if x.endswith(".json")]

# Train-Test-Split
if len(images) < 2 or len(annotations) < 2:
    print("Nicht genügend Datenpunkte für den Split vorhanden.")
    exit()

train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.3, random_state=1)

# Validierungs-Split
val_images, test_images, val_annotations, test_annotations = train_test_split(test_images, test_annotations, test_size=0.5, random_state=1)

# Funktion zum Kopieren von Dateien
def copy_files_to_folder(file_list, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    for file_path in file_list:
        file_name = os.path.basename(file_path)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy(file_path, destination_path)

# Funktion zur Konvertierung der Labels ins YOLOv5-Format
def convert_labels_to_yolov5_format(json_path, image_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    class_mapping = {
        "face_no_mask": 0,
        "face_with_mask": 1,
        "mask_surgical": 2,
        "hat": 3,
        "eyeglasses": 4,
        "face_other_covering": 5,
        "face_with_mask_incorrect": 6,
        "mask_colorful": 7,
        "helmet": 8,
        "sunglasses": 9,
        "scarf_bandana": 10,
        "hair_net": 11,
        "goggles": 12,
        "face_shield": 13,
        "hijab_niqab": 14,
        "turban": 15,
        "balaclava_ski_mask": 16,
        "gas_mask": 17,
        "hood": 18,
        "other": 19
    }

    # Erstelle YOLOv5-Annotationen
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    yolo_annotations = []
    for bbox in json_data["Annotations"]:
        x_min, y_min, x_max, y_max = bbox["BoundingBox"]
        class_name = bbox["classname"]

        if class_name not in class_mapping:
            raise ValueError(f"Ungültige Klasse: {class_name}")

        class_index = class_mapping[class_name]

        # Normalisierung der Koordinaten
        x_center = (x_min + x_max) / (2 * image_width)
        y_center = (y_min + y_max) / (2 * image_height)
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_annotation = f"{class_index} {x_center} {y_center} {width} {height}"
        yolo_annotations.append(yolo_annotation)

    image_filename = os.path.splitext(os.path.basename(image_path))[0]
    output_filename = f"{image_filename}.txt"
    output_path = os.path.join(output_dir, output_filename)

    with open(output_path, 'w') as f:
        for annotation in yolo_annotations:
            f.write(annotation + '\n')

    # Erstelle classes.txt-Datei
    classes_file_path = os.path.join(output_dir, "classes.txt")
    with open(classes_file_path, 'w') as f:
        for class_name, class_index in class_mapping.items():
            f.write(f"{class_index} {class_name}\n")




# Bilder in Trainings-, Validierungs- und Testordner kopieren
train_image_output_dir = os.path.join(image_output_dir, "train")
val_image_output_dir = os.path.join(image_output_dir, "val")
test_image_output_dir = os.path.join(image_output_dir, "test")
copy_files_to_folder(train_images, train_image_output_dir)
copy_files_to_folder(val_images, val_image_output_dir)
copy_files_to_folder(test_images, test_image_output_dir)

# Labels in den entsprechenden Ordnern kopieren und ins YOLOv5-Format konvertieren
train_labels_output_dir = os.path.join(labels_output_dir, "train")
val_labels_output_dir = os.path.join(labels_output_dir, "val")
test_labels_output_dir = os.path.join(labels_output_dir, "test")
copy_files_to_folder(train_annotations, train_labels_output_dir)
copy_files_to_folder(val_annotations, val_labels_output_dir)
copy_files_to_folder(test_annotations, test_labels_output_dir)

# Labels ins YOLOv5-Format konvertieren
for json_path, image_path in zip(train_annotations, train_images):
    convert_labels_to_yolov5_format(json_path, image_path, train_labels_output_dir)

for json_path, image_path in zip(val_annotations, val_images):
    convert_labels_to_yolov5_format(json_path, image_path, val_labels_output_dir)

for json_path, image_path in zip(test_annotations, test_images):
    convert_labels_to_yolov5_format(json_path, image_path, test_labels_output_dir)

print("Split, Kopieren und Konvertieren erfolgreich.")


Split, Kopieren und Konvertieren erfolgreich.


In [21]:
#JSON-Dateien aus yolov5/dataset löschen
def delete_json_files(folders):
    for folder in folders:
        if not os.path.exists(folder):
            continue
        
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            
            if filename.endswith(".json"):
                os.remove(file_path)

folders = [
    "yolov5/dataset/labels/train",
    "yolov5/dataset/labels/test",
    "yolov5/dataset/labels/val",
]

delete_json_files(folders)

### Training von YOLOv5 mit WandB-Integration und Upload des Datasets

conda install pytorch=1.9.0 cudatoolkit=11.1 -c pytorch -c conda-forge  
damit es läuft
torchvision               0.10.0+cu111

In [22]:
import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3080', major=8, minor=6, total_memory=10239MB, multi_processor_count=68)


In [23]:
import wandb
wandb.login()

True

In [24]:
%cd yolov5

c:\Users\Saniye\Documents\GitHub\Deep-Vision-sta\YOLO\yolov5


In [31]:
!python train.py --data dataset/data.yaml --epochs 10 --weights '' --cfg yolov5s.yaml  --batch-size 6 --upload_dataset --bbox_interval 1 --img 640

Traceback (most recent call last):
  File "c:\Users\Saniye\Documents\GitHub\Deep-Vision-sta\YOLO\yolov5\train.py", line 35, in <module>
    import torch
  File "c:\Users\Saniye\anaconda3\envs\DEEP_VISION_KLONE4\lib\site-packages\torch\__init__.py", line 124, in <module>
    raise err
OSError: [WinError 1455] Die Auslagerungsdatei ist zu klein, um diesen Vorgang durchzuführen. Error loading "c:\Users\Saniye\anaconda3\envs\DEEP_VISION_KLONE4\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.
